# Test low pass after LOWESS
- recompute x, y afterward
- recompte velocities

## TO DO ADD GEO OPTION

In [35]:
import xarray as xr
import pandas as pd
import numpy as np
import scipy as sc

import hvplot.xarray
import hvplot.pandas

import matplotlib.pyplot as plt
from cycler import cycler

from sstats import signals as sg
from sstats import sigp as sigp
from sstats import tseries as ts
from sstats import get_cmap_colors

import hvplot.xarray
import hvplot.pandas
import holoviews as hv

import pynsitu as pyn
import lib as lib
from lib import KEYS, raw_dir, images_dir
import os
from glob import glob

import synthetic_traj as st

from synthetic_traj import (
    synthetic_traj,
    noise_irregular_sampling,
    ref_case,
    typical_case,
)

___________
# Reference case

In [2]:
N = 40
position_noise = 20
ntype = "white_noise"
offset_type = "svp_scripps_10"
spectral_diff = False
print(ref_case)

{'T': 5, 'U_low': 0.3, 'U_ni': 0.2, 'U_2': 0, 'U_1': 0, 'tau_eta': 0.1, 'n_layers': 5, 'spectral_diff': False}


In [3]:
ref_case["spectral_diff"] = spectral_diff

___________
# LOWESS + LOWPAS
Parameters : degree = 2 or 3

In [4]:
DF = dict()

# TRUE
# t = (50, 1/24/60)
t = (
    50,
    "1min",
)  # use it instead of (50, 1/24/60 because otherwise not regularly sampled

dst = synthetic_traj(t, N, **ref_case)  # u,v, ax, ay computed
dft = st.dataset2dataframe(dst).rename(columns={"draw": "id"})
DF["True_1min"] = dft

# OBSERVED

dso = noise_irregular_sampling(
    dst, t, position_noise, ntype=ntype, offset_type=offset_type, istart=4067
)
dfo = st.dataset2dataframe(dso).rename(columns={"draw": "id"})
dfo = dfo.groupby("id").apply(pyn.geo.compute_dt, time="index")

DF["Observed"] = dfo

# SMOOTHED
dt_smooth = "30min"
dfl = pyn.drifters.smooth_all(
    dfo, "lowess", dt_smooth, spectral_diff=spectral_diff, geo=False
)

DF["LOWESS"] = dfl

# Interpolated True
dsti = dst.sel(time=dfl[dfl.id == 0].index.values)
dsti["dt"] = dsti.time.diff("time") / pd.Timedelta("1s")
dfti = st.dataset2dataframe(dsti).rename(columns={"draw": "id"})
dfti = dfti.groupby("id").apply(
    pyn.geo.compute_velocities,
    time="index",
    distance="xy",
    names=("u", "v", "U"),
    fill_startend=True,
    centered=True,
    keep_dt=True,
)
dfti = dfti.groupby("id").apply(
    pyn.geo.compute_accelerations,
    from_=("xy", "x", "y"),
    names=("ax", "ay", "Axy"),
    keep_dt=True,
)

DF["True_" + dt_smooth] = dfti

centred diff
centred diff
22186.495695337686
4067


In [142]:
from pynsitu.drifters import compute_accelerations


def low_pass(df, T=20, cutoff=4):
    from scipy.signal import filtfilt
    from scipy.integrate import cumulative_trapezoid
    from scipy.optimize import minimize

    # coefficients
    dt = df.dt.mean() / 3600 / 24  # in days
    taps = pyn.tseries.generate_filter(band="low", dt=dt, T=T, bandwidth=cutoff)
    dff = df[["u", "v"]]
    # apply filter
    dff["u"] = filtfilt(taps, 1, df.u.values)
    dff["v"] = filtfilt(taps, 1, df.v.values)

    # recompute position
    # ms_x, ms_y = (df.x**2).mean(), (df.y**2).mean()
    x_cum = cumulative_trapezoid(dff.u, dx=df_.dt.mean(), initial=0)
    y_cum = cumulative_trapezoid(dff.v, dx=df_.dt.mean(), initial=0)

    def msx_difference(x_0):
        return ((df.x - x_0 - x_cum) ** 2).mean()

    def msy_difference(y_0):
        return ((df.y - y_0 - y_cum) ** 2).mean()

    x_0 = minimize(msx_difference, df.x[0]).x
    y_0 = minimize(msy_difference, df.y[0]).x

    dff["x"] = x_0 + x_cum
    dff["y"] = y_0 + y_cum

    # recompute acceleration
    compute_accelerations(
        dff,
        from_=("xy", "x", "y"),
        names=("ax", "ay", "Axy"),
        centered_velocity=True,
        time="index",
        fill_startend=True,
        inplace=True,
        keep_dt=False,
    )
    compute_accelerations(
        dff,
        from_=("velocities", "u", "v"),
        names=("au", "av", "Auv"),
        centered_velocity=True,
        time="index",
        fill_startend=True,
        inplace=True,
        keep_dt=True,
    )
    return dff


df_ = dfl[dfl.id == 0]
dff_ = low_pass(df_)

In [143]:
dff_

,u,v,x,y,ax,ay,Axy,dt,au,av,Auv
time,,,,,,,,,,,
2000-01-01 00:10:00,0.341300,-0.355054,519.659695,-200.988185,-0.000016,-4.287120e-06,0.000017,1800.0,-0.000016,-4.287120e-06,0.000017
2000-01-01 00:40:00,0.314074,-0.363888,1109.496536,-848.036601,-0.000016,-4.287120e-06,0.000017,1800.0,-0.000016,-4.287120e-06,0.000017
2000-01-01 01:10:00,0.283162,-0.370488,1647.009106,-1508.975285,-0.000019,-2.467687e-06,0.000019,1800.0,-0.000019,-2.467687e-06,0.000019
2000-01-01 01:40:00,0.245646,-0.372772,2122.936193,-2177.909273,-0.000023,4.273168e-07,0.000023,1800.0,-0.000023,4.273168e-07,0.000023
2000-01-01 02:10:00,0.199987,-0.368950,2524.005689,-2845.458754,-0.000028,4.205374e-06,0.000028,1800.0,-0.000028,4.205374e-06,0.000028
...,...,...,...,...,...,...,...,...,...,...,...
2000-02-19 21:40:00,0.551621,-0.084112,-74136.210260,-16588.828141,0.000003,-1.959927e-05,0.000020,1800.0,0.000003,-1.959927e-05,0.000020
2000-02-19 22:10:00,0.552496,-0.116645,-73142.505232,-16769.509316,-0.000002,-1.637304e-05,0.000016,1800.0,-0.000002,-1.637304e-05,0.000016
2000-02-19 22:40:00,0.545760,-0.143055,-72154.075001,-17003.239124,-0.000005,-1.323865e-05,0.000014,1800.0,-0.000005,-1.323865e-05,0.000014


In [146]:
df_.x.hvplot() * dff_.x.hvplot()

:Overlay
   .Curve.X.I  :Curve   [time]   (x)
   .Curve.X.II :Curve   [time]   (x)